In [ ]:
from django.contrib.auth.models import AbstractUser
from django.db import models
from django.utils.translation import gettext_lazy as _


class Permission(models.Model):
    """
    Model to represent permissions.
    """

    name = models.CharField(max_length=100, unique=True)
    description = models.TextField()

    def __str__(self):
        return self.name


class RoleManager(models.Manager):
    def with_permissions(self):
        """
        Returns roles with associated permissions.
        """
        return self.get_queryset().select_related("permissions")


class Role(models.Model):
    """
    Model for roles in the system.
    """

    name = models.CharField(max_length=15, unique=True)
    description = models.TextField()
    permissions = models.ManyToManyField(Permission, related_name="roles")
    objects = RoleManager()

    def __str__(self):
        return self.name


class CustomUser(AbstractUser):
    """
    Custom user model with additional fields and methods.
    """

    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)

    def has_specific_permission(self, permission_name):
        """
        Check if the user has a specific permission.
        """
        return self.user_permissions.filter(name=permission_name).exists() or (
            self.role and self.role.permissions.filter(name=permission_name).exists()
        )


class UserProfile(models.Model):
    """
    Model for user profiles with additional information.
    """

    user = models.OneToOneField(CustomUser, on_delete=models.CASCADE)
    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(
        upload_to="profile_pics/", blank=True, null=True
    )
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)

    def __str__(self):
        return f"{self.user.username} - {self.role}"

    class Meta:
        permissions = [
            ("is_admin", "Can access admin functionalities"),
            ("is_translator", "Can perform translation tasks"),
            ("is_project_manager", "Can manage projects"),
            ("is_developer", "Can perform development tasks"),
            ("is_client", "Is a client"),
        ]


class Client(models.Model):
    user_profile = models.OneToOneField(UserProfile, on_delete=models.CASCADE)
    secondary_email = models.EmailField(blank=True, null=True)
    secondary_phone_number = models.CharField(max_length=15, blank=True, null=True)
    linkedin_profile = models.URLField(blank=True)
    twitter_handle = models.CharField(max_length=50, blank=True)
    industry = models.CharField(max_length=50, blank=True)
    company_size = models.PositiveIntegerField(null=True, blank=True)

    def __str__(self):
        return self.user_profile.user.username


class Interaction(models.Model):
    client = models.ForeignKey(
        Client, on_delete=models.CASCADE, related_name="interactions"
    )
    date = models.DateTimeField()
    interaction_type = models.CharField(
        max_length=20,
        choices=[
            ("meeting", "Meeting"),
            ("call", "Call"),
            ("email", "Email"),
            ("presentation", "Presentation"),
            ("other", "Other"),
        ],
    )
    notes = models.TextField()

    def __str__(self):
        return f"{self.client.user_profile.user.username} - {self.get_interaction_type_display()} Interaction"


class Task(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    assignee = models.ForeignKey(
        UserProfile, on_delete=models.CASCADE, related_name="assigned_tasks"
    )
    due_date = models.DateField()
    status = models.CharField(
        max_length=20,
        choices=[
            ("todo", "To Do"),
            ("in_progress", "In Progress"),
            ("completed", "Completed"),
        ],
        default="todo",
    )
    priority = models.PositiveIntegerField(
        choices=[
            (1, "Low"),
            (2, "Medium"),
            (3, "High"),
        ],
        default=2,
    )
    project = models.ForeignKey(
        "Project", on_delete=models.CASCADE, related_name="tasks"
    )

    def __str__(self):
        return self.title


class TimeEntry(models.Model):
    user = models.ForeignKey(UserProfile, on_delete=models.CASCADE)
    task = models.ForeignKey(Task, on_delete=models.CASCADE, null=True, blank=True)
    project = models.ForeignKey("Project", on_delete=models.CASCADE)
    start_time = models.DateTimeField()
    end_time = models.DateTimeField()
    hours_worked = models.DecimalField(max_digits=5, decimal_places=2)

    def __str__(self):
        return f"{self.user.user.username} - {self.project.title} - {self.start_time} to {self.end_time}"


class Expense(models.Model):
    amount = models.DecimalField(max_digits=10, decimal_places=2)
    category = models.CharField(max_length=50)
    description = models.TextField()
    date = models.DateField()
    project = models.ForeignKey("Project", on_delete=models.CASCADE)

    def __str__(self):
        return f"{self.amount} - {self.category} - {self.project.title} - {self.date}"


class Project(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    start_date = models.DateField()
    end_date = models.DateField(null=True, blank=True)
    client = models.ForeignKey(
        Client, on_delete=models.CASCADE, related_name="projects"
    )
    team_members = models.ManyToManyField(UserProfile, related_name="projects_assigned")
    budget = models.DecimalField(max_digits=10, decimal_places=2, null=True, blank=True)
    actual_cost = models.DecimalField(
        max_digits=10, decimal_places=2, null=True, blank=True
    )
    status = models.CharField(
        max_length=20,
        choices=[
            ("pending", "Pending"),
            ("in_progress", "In Progress"),
            ("completed", "Completed"),
        ],
        default="pending",
    )
    priority = models.PositiveIntegerField(default=1)
    is_billable = models.BooleanField(default=True)
    notes = models.TextField(blank=True)

    def __str__(self):
        return self.title

In [ ]:
from django.contrib.auth.models import AbstractUser
from django.db import models
from django.utils.translation import gettext_lazy as _
from django.core.validators import MinValueValidator


class Permission(models.Model):
    """
    Model to represent permissions.
    """

    name = models.CharField(max_length=100, unique=True)
    description = models.TextField()

    def __str__(self):
        return self.name


class RoleManager(models.Manager):
    def with_permissions(self):
        """
        Returns roles with associated permissions.
        """
        return self.get_queryset().select_related("permissions")


class Role(models.Model):
    """
    Model for roles in the system.
    """

    name = models.CharField(max_length=15, unique=True)
    description = models.TextField()
    permissions = models.ManyToManyField(Permission, related_name="roles")
    objects = RoleManager()

    def __str__(self):
        return self.name


class CustomUserManager(models.Manager):
    def with_role(self):
        """
        Returns users with associated roles.
        """
        return self.get_queryset().select_related("role")


class CustomUser(AbstractUser):
    """
    Custom user model with additional fields and methods.
    """

    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)
    objects = CustomUserManager()

    def has_specific_permission(self, permission_name):
        """
        Check if the user has a specific permission.
        """
        return self.user_permissions.filter(name=permission_name).exists() or (
            self.role and self.role.permissions.filter(name=permission_name).exists()
        )


class UserProfile(models.Model):
    """
    Model for user profiles with additional information.
    """

    user = models.OneToOneField(CustomUser, on_delete=models.CASCADE)
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(
        upload_to="profile_pics/", blank=True, null=True
    )
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)

    def __str__(self):
        return self.user.username


class Client(models.Model):
    user_profile = models.OneToOneField(UserProfile, on_delete=models.CASCADE)
    secondary_email = models.EmailField(blank=True, null=True)
    secondary_phone_number = models.CharField(max_length=15, blank=True, null=True)
    linkedin_profile = models.URLField(blank=True)
    twitter_handle = models.CharField(max_length=50, blank=True)
    industry = models.CharField(max_length=50, blank=True)
    company_size = models.PositiveIntegerField(null=True, blank=True)

    def __str__(self):
        return self.user_profile.user.username


class InteractionManager(models.Manager):
    def for_client(self, client):
        """
        Returns interactions associated with a specific client.
        """
        return self.filter(client=client)


class Interaction(models.Model):
    client = models.ForeignKey(
        Client, on_delete=models.CASCADE, related_name="interactions"
    )
    date = models.DateTimeField()
    interaction_type = models.CharField(
        max_length=20,
        choices=[
            ("meeting", "Meeting"),
            ("call", "Call"),
            ("email", "Email"),
            ("presentation", "Presentation"),
            ("other", "Other"),
        ],
    )
    notes = models.TextField()
    objects = InteractionManager()

    def __str__(self):
        return f"{self.client.user_profile.user.username} - {self.get_interaction_type_display()} Interaction"


class TaskManager(models.Manager):
    def for_assignee(self, assignee):
        """
        Returns tasks assigned to a specific user.
        """
        return self.filter(assignee=assignee)


class Task(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    assignee = models.ForeignKey(
        UserProfile, on_delete=models.CASCADE, related_name="assigned_tasks"
    )
    due_date = models.DateField()
    status = models.CharField(
        max_length=20,
        choices=[
            ("todo", "To Do"),
            ("in_progress", "In Progress"),
            ("completed", "Completed"),
        ],
        default="todo",
    )
    priority = models.PositiveIntegerField(
        choices=[
            (1, "Low"),
            (2, "Medium"),
            (3, "High"),
        ],
        default=2,
    )
    project = models.ForeignKey(
        "Project", on_delete=models.CASCADE, related_name="tasks"
    )
    objects = TaskManager()

    def __str__(self):
        return self.title


class TimeEntry(models.Model):
    user = models.ForeignKey(UserProfile, on_delete=models.CASCADE)
    task = models.ForeignKey(Task, on_delete=models.CASCADE, null=True, blank=True)
    project = models.ForeignKey("Project", on_delete=models.CASCADE)
    start_time = models.DateTimeField()
    end_time = models.DateTimeField()
    hours_worked = models.DecimalField(max_digits=5, decimal_places=2)

    def __str__(self):
        return f"{self.user.user.username} - {self.project.title} - {self.start_time} to {self.end_time}"


class Expense(models.Model):
    amount = models.DecimalField(max_digits=10, decimal_places=2)
    category = models.CharField(max_length=50)
    description = models.TextField()
    date = models.DateField()
    project = models.ForeignKey("Project", on_delete=models.CASCADE)

    def __str__(self):
        return f"{self.amount} - {self.category} - {self.project.title} - {self.date}"


class Project(models.Model):
    title = models.CharField(max_length=100, unique=True)
    description = models.CharField(max_length=500)
    notes = models.TextField(max_length=1000)
    start_date = models.DateField()
    end_date = models.DateField(null=True, blank=True)
    client = models.ForeignKey(
        Client, on_delete=models.PROTECT, related_name="projects"
    )  # Change on_delete behavior
    team_members = models.ManyToManyField(UserProfile, related_name="projects_assigned")
    budget = models.DecimalField(
        max_digits=10,
        decimal_places=2,
        null=True,
        blank=True,
        validators=[MinValueValidator(0)],
    )
    actual_cost = models.DecimalField(
        max_digits=10,
        decimal_places=2,
        null=True,
        blank=True,
        validators=[MinValueValidator(0)],
    )
    status = models.CharField(
        max_length=20,
        choices=[
            ("pending", _("Pending")),
            ("in_progress", _("In Progress")),
            ("completed", _("Completed")),
        ],
        default="pending",
    )
    priority = models.PositiveIntegerField(default=1, validators=[MinValueValidator(1)])
    is_billable = models.BooleanField(default=True)

    def __str__(self):
        return self.title

    # Add additional model-level validation if needed
    def clean(self):
        if self.start_date and self.end_date and self.start_date > self.end_date:
            raise models.ValidationError(_("Start date cannot be after end date."))

        # Add any other custom validation logic here

    def duration(self):
        """
        Calculate the duration of the project in days.
        """
        if self.end_date:
            return (self.end_date - self.start_date).days
        else:
            return None

    def estimated_remaining_budget(self):
        """
        Estimate the remaining budget based on actual cost and budget.
        """
        if self.actual_cost is not None and self.budget is not None:
            return self.budget - self.actual_cost
        else:
            return None

    def generate_report(self):
        """
        Generate a report for the project.
        """
        report = f"Project: {self.title}\n"
        report += f"Description: {self.description}\n"
        report += f"Start Date: {self.start_date}\n"
        report += f"End Date: {self.end_date}\n"
        report += f"Status: {self.get_status_display()}\n"
        report += f"Duration: {self.duration()} days\n"
        report += f"Budget: {self.budget}\n"
        report += f"Actual Cost: {self.actual_cost}\n"
        report += f"Remaining Budget: {self.estimated_remaining_budget()}\n"
        # Add more details as needed
        return report

In [ ]:
from django.contrib.auth.models import Permission
from django.contrib.contenttypes.models import ContentType
from django.db import models

class CustomPermissions:
    APPROVE_EXPENSE = 'approve_expense'
    MANAGE_BUDGET = 'manage_budget'

# Define custom permissions
Permission.objects.create(
    codename=CustomPermissions.APPROVE_EXPENSE,
    name='Can approve expenses',
    content_type=ContentType.objects.get_for_model(Expense),
)
Permission.objects.create(
    codename=CustomPermissions.MANAGE_BUDGET,
    name='Can manage project budgets',
    content_type=ContentType.objects.get_for_model(Project),
)

In [ ]:
from django.contrib.auth.models import AbstractUser
from django.db import models
from django.utils.translation import gettext_lazy as _
from django.core.validators import MinValueValidator


class Permission(models.Model):
    name = models.CharField(max_length=100, unique=True)
    description = models.TextField()

    def __str__(self):
        return self.name


class RoleManager(models.Manager):
    def with_permissions(self):
        return self.get_queryset().select_related("permissions")


class Role(models.Model):
    name = models.CharField(max_length=15, unique=True)
    description = models.TextField()
    permissions = models.ManyToManyField(Permission, related_name="roles")
    objects = RoleManager()

    def __str__(self):
        return self.name


class CustomUserManager(models.Manager):
    def with_role(self):
        return self.get_queryset().select_related("role")


class CustomUser(AbstractUser):
    role = models.ForeignKey(Role, on_delete=models.SET_NULL, null=True)
    objects = CustomUserManager()

    def has_specific_permission(self, permission_name):
        return self.user_permissions.filter(name=permission_name).exists() or (
            self.role and self.role.permissions.filter(name=permission_name).exists()
        )


class UserProfile(models.Model):
    user = models.OneToOneField(CustomUser, on_delete=models.CASCADE)
    bio = models.TextField(blank=True)
    profile_picture = models.ImageField(
        upload_to="profile_pics/", blank=True, null=True
    )
    phone_number = models.CharField(max_length=15, blank=True, null=True)
    address = models.TextField(blank=True)

    def __str__(self):
        return self.user.username


class Client(models.Model):
    user_profile = models.OneToOneField(UserProfile, on_delete=models.CASCADE)
    secondary_email = models.EmailField(blank=True, null=True)
    secondary_phone_number = models.CharField(max_length=15, blank=True, null=True)
    linkedin_profile = models.URLField(blank=True)
    twitter_handle = models.CharField(max_length=50, blank=True)
    industry = models.CharField(max_length=50, blank=True)
    company_size = models.PositiveIntegerField(null=True, blank=True)

    def __str__(self):
        return self.user_profile.user.username


class Interaction(models.Model):
    client = models.ForeignKey(
        Client, on_delete=models.CASCADE, related_name="interactions"
    )
    date = models.DateTimeField()
    interaction_type = models.CharField(
        max_length=20,
        choices=[
            ("meeting", "Meeting"),
            ("call", "Call"),
            ("email", "Email"),
            ("presentation", "Presentation"),
            ("other", "Other"),
        ],
    )
    notes = models.TextField()

    def __str__(self):
        return f"{self.client.user_profile.user.username} - {self.get_interaction_type_display()} Interaction"


class Task(models.Model):
    title = models.CharField(max_length=200)
    description = models.TextField()
    assignee = models.ForeignKey(
        UserProfile, on_delete=models.CASCADE, related_name="assigned_tasks"
    )
    due_date = models.DateField()
    status = models.CharField(
        max_length=20,
        choices=[
            ("todo", "To Do"),
            ("in_progress", "In Progress"),
            ("completed", "Completed"),
        ],
        default="todo",
    )
    priority = models.PositiveIntegerField(
        choices=[
            (1, "Low"),
            (2, "Medium"),
            (3, "High"),
        ],
        default=2,
    )
    project = models.ForeignKey(
        "Project", on_delete=models.CASCADE, related_name="tasks"
    )

    def __str__(self):
        return self.title


class TimeEntry(models.Model):
    user = models.ForeignKey(CustomUser, on_delete=models.CASCADE)
    task = models.ForeignKey(Task, on_delete=models.CASCADE)
    project = models.ForeignKey("Project", on_delete=models.CASCADE)
    start_time = models.DateTimeField()
    end_time = models.DateTimeField()
    hours_worked = models.DecimalField(max_digits=5, decimal_places=2)

    def __str__(self):
        return f"{self.user.username} - {self.project.title} - {self.start_time} to {self.end_time}"


class Expense(models.Model):
    amount = models.DecimalField(max_digits=10, decimal_places=2)
    category = models.CharField(max_length=50)
    description = models.TextField()
    date = models.DateField()
    project = models.ForeignKey("Project", on_delete=models.CASCADE)

    def __str__(self):
        return f"{self.amount} - {self.category} - {self.project.title} - {self.date}"


class Project(models.Model):
    title = models.CharField(max_length=100, unique=True)
    description = models.CharField(max_length=500)
    notes = models.TextField(max_length=1000)
    start_date = models.DateField()
    end_date = models.DateField(null=True, blank=True)
    client = models.ForeignKey(
        Client, on_delete=models.PROTECT, related_name="projects"
    )
    team_members = models.ManyToManyField(UserProfile, related_name="projects_assigned")
    budget = models.DecimalField(
        max_digits=10,
        decimal_places=2,
        null=True,
        blank=True,
        validators=[MinValueValidator(0)],
    )
    actual_cost = models.DecimalField(
        max_digits=10,
        decimal_places=2,
        null=True,
        blank=True,
        validators=[MinValueValidator(0)],
    )
    status = models.CharField(
        max_length=20,
        choices=[
            ("pending", _("Pending")),
            ("in_progress", _("In Progress")),
            ("completed", _("Completed")),
        ],
        default="pending",
    )
    priority = models.PositiveIntegerField(default=1, validators=[MinValueValidator(1)])
    is_billable = models.BooleanField(default=True)

    def __str__(self):
        return self.title

    def clean(self):
        if self.start_date and self.end_date and self.start_date > self.end_date:
            raise models.ValidationError(_("Start date cannot be after end date."))

    def duration(self):
        if self.end_date:
            return (self.end_date - self.start_date).days
        else:
            return None

    def estimated_remaining_budget(self):
        if self.actual_cost is not None and self.budget is not None:
            return self.budget - self.actual_cost
        else:
            return None

    def generate_report(self):
        report = f"Project: {self.title}\n"
        report += f"Description: {self.description}\n"
        report += f"Start Date: {self.start_date}\n"
        report += f"End Date: {self.end_date}\n"
        report += f"Status: {self.get_status_display()}\n"
        report += f"Duration: {self.duration()} days\n"
        report += f"Budget: {self.budget}\n"
        report += f"Actual Cost: {self.actual_cost}\n"
        report += f"Remaining Budget: {self.estimated_remaining_budget()}\n"
        return report
